# Orchestration with GenAI Hub

This notebook demonstrates setting up data masking and content filtering, configuring an orchestration pipeline, and querying multiple LLM models with GenAI Hub.

Installing the required packages

In [ ]:
pip install hdbcli pandas generative-ai-hub-sdk[all] langchain --break-system-packages

The code imports required libraries, reads credentials from a creds.json file, and sets environment variables for authentication and API access

In [1]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from enum import Enum
 
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "llm-deployed" 
    }

    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

https://anuragv2-39wjy902.authentication.eu10.hana.ondemand.com/oauth/token
sb-adcd4907-f1a7-462d-ba8e-646390ee4185!b398425|aicore!b540
107d3e9b-9a41-4b30-9b24-a091a45956cd$VmYxjzammFm50xkj1O37HmzgX3maoNrwfrlm99qUhi0=
https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
llm-deployed


## Initializing the Orchestration Service

⚠️Before using the SDK, you need to set up a virtual deployment of the Orchestration Service. Once deployed, you'll have access to a unique endpoint URL (deploymentUrl).

In [2]:
YOUR_API_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d414014d74d0eb17"


# Import Libraries
Here, we load essential libraries to manage the orchestration setup and handle data masking and content filtering for queries.


In [3]:
# Import necessary libraries for data masking, content filtering, and orchestration
import pandas as pd
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter
from gen_ai_hub.orchestration.models.config import OrchestrationConfig, Template
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.service import OrchestrationService


# Step 1: Templating

Explanation of Templating Code
This code defines a template for an AI assistant using orchestration configuration. The `Template` object is set up with system and user messages to guide the assistant’s response behavior. 

Key Components:
- **SystemMessage**: Sets a predefined instruction for the AI assistant. This message typically includes the assistant's role and any specific guidelines it should follow.
- **UserMessage**: Represents the user's input and how it is structured in the conversation.
  
In this revised prompt, only queries are passed to the assistant without any additional context. The AI is expected to respond based solely on the provided input.


In [4]:
template = Template(
    messages=[
            SystemMessage(
                """You are an AI assistant. You are provided with the top RAG results (Context) from a Reddit post. 
                Use the following Context {{?context}} to answer the User_input {{?text}} by selecting the most relevant response. 
                Ensure you provide an answer **exactly as it appears** in the Context, with no alterations or variations. 
                Do not generate any new information.

                NOTE:
                Use only the provided Context {{?context}} to answer User_input {{?text}}. 
                If no answer found reply 'I don't know'."""
            ),
            UserMessage("User_input: {{?text}}, Context: {{?context}}")
        ]
    )

# Step 2: Setup Data Masking
 This step involves configuring data masking to anonymize or pseudonymize sensitive information (e.g., email, phone, person names).
 * `SAPDataPrivacyIntegration` is used to specify which types of data should be masked.
 * Choose between `ANONYMIZATION` or `PSEUDONYMIZATION` as needed.


In [5]:
# Configure data masking settings
data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # Choose MaskingMethod.PSEUDONYMIZATION if pseudonymization is needed
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)


# Step 3: Define Content Filters
 Setting up filters for specific content types ensures that the query inputs and outputs are within acceptable limits for the intended application.
 * Input and output filters are configured to exclude certain types of content, such as violence or self-harm.


In [6]:
# Configure input and output content filters
input_filter = AzureContentFilter(hate=0, sexual=4, self_harm=0, violence=2)
output_filter = AzureContentFilter(hate=0, sexual=4, self_harm=0, violence=2)


# Step 4: Create the Orchestration Config Function
 This function configures orchestration for a given query, setting up the template, model, and content filters.
 Each model uses the same orchestration setup and applies data masking and content filtering to the responses.


In [7]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.service import OrchestrationService

# Define orchestration function
def Orchestration_run(query, context, model_name):
    # Configure the orchestration with templating and model
    config = OrchestrationConfig(
        template=template,
        llm=LLM(name=model_name),
        input_filters=[input_filter],
        output_filters=[output_filter]
    )
    
    orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)
    
    # Run orchestration with the provided query and context
    result = orchestration_service.run(
        config=config,
        template_values=[
            TemplateValue(name="text", value=query),
            TemplateValue(name="context", value=context)
        ]
    )
    
    masked_result = result.orchestration_result.choices[0].message.content
    return masked_result


# Step 5: Querying LLM Models 
In this step, we will modify the orchestration code to query multiple LLM models directly. The objective is to send predefined questions to each model and receive their responses 


In [8]:
# Define models to query
models = ["gpt-4o", "gemini-1.5-pro", "meta--llama3-70b-instruct"]


Define the Questions to Query

We will define a list of questions that we want to ask the LLM models. 


In [9]:
# Define questions to be queried
questions = [
    "What are the primary benefits of using SAP AI Core for managing AI scenarios?", 
    "How does SAP AI Core facilitate the integration of AI models with other SAP solutions?", 
    "What improvements does the latest version of SAP AI Core offer for developers?", 
    "Why is prompt lifecycle management essential in the context of generative AI?", 
    "In what ways can the Generative AI Hub assist in the development of AI models?", 
    "How do APIs in SAP AI Core contribute to enhanced machine learning functionalities?", 
    "What methods can be employed to optimize the performance of AI models in SAP AI Core?", 
    "Can you describe the process for managing datasets and model files using the Artifacts feature?", 
    "What strategies can ensure the effective deployment and maintenance of AI models within SAP AI Core?", 
    "What role does the AI API play in monitoring the performance metrics of AI models?"
]


Setting Up Context for the Orchestration Function

In [10]:
context = """Title: Understanding the Basics of SAP AI Core
Query: What is SAP AI Core and what are its main functions?
Answer: SAP AI Core is a service within the SAP Business Technology Platform that manages AI assets in a 
standardized and scalable manner. It supports the execution and operations of AI scenarios, allowing for 
seamless integration with SAP solutions. Its main functions include lifecycle management of AI scenarios, 
data-driven decision-making, and automation of tasks using machine learning capabilities.
Title: Latest Updates in SAP AI Core
Query: What new features were introduced in the latest version of SAP AI Core?
Answer: The latest version of SAP AI Core includes enhancements in generative AI capabilities, improved 
integration with open-source frameworks, and better support for prompt lifecycle management. These 
updates aim to streamline the development and deployment of AI models, making it easier for businesses to 
leverage AI technologies.
Title: How to Connect Your Data to SAP AI Core
Query: What steps should I follow to connect my data with SAP AI Core?
Answer: To connect your data with SAP AI Core, you can use cloud storage to store AI assets such as datasets 
and model files. You will reference these assets using the Artifacts feature in SAP AI Core, which allows for 
efficient management and retrieval of your data.
Title: Training Your AI Model in SAP AI Core
Query: How can I effectively train my AI model using SAP AI Core?
Answer: Training your AI model in SAP AI Core involves executing a training workflow that allows you to input 
your data and configure the model parameters. The platform supports various machine learning frameworks, 
enabling you to optimize your model for better performance.
Title: Exploring the Generative AI Hub
Query: What is the Generative AI Hub and how can I use it?
Answer: The Generative AI Hub is a component of SAP AI Core that allows users to experiment with generative 
AI models. It provides tools for prompt experimentation, management, and administrative functions, enabling 
users to create and run natural language prompts effectively.
Title: The Role of APIs in SAP AI Core
Query: How do APIs enhance the functionality of SAP AI Core?
Answer: APIs in SAP AI Core facilitate the integration of AI capabilities into applications, allowing developers to 
access machine learning functionalities seamlessly. They provide standardized methods for interacting with AI 
models, making it easier to implement AI solutions in various business processes.
Title: Metrics Tracking in AI Core
Query: How can I track the performance metrics of my AI models in SAP AI Core?
Answer: You can track performance metrics in SAP AI Core using the AI API, which allows you to monitor 
various aspects of your AI models, such as accuracy, response time, and resource usage. This tracking helps in 
optimizing model performance and ensuring effective deployment.
Title: Advanced Features of SAP AI Core
Query: What advanced features does SAP AI Core offer for AI development?
Answer: SAP AI Core offers advanced features such as AI Content as a Service, which enables users to provide 
their AI content on the Service Marketplace. Additionally, it includes orchestration capabilities that combine 
content generation with essential business functions, enhancing the overall AI development process.
Title: Best Practices for Using Generative AI
Query: What are some best practices for utilizing generative AI models in SAP AI Core?
Answer: Best practices for using generative AI models include clearly defining your prompts, experimenting 
with different model parameters, and continuously monitoring the output for quality. It's also important to 
manage prompt lifecycles effectively to ensure optimal performance and relevance.
Title: What is AI Content as a Service?
Query: Can someone explain AI Content as a Service in SAP AI Core?
Answer: AI Content as a Service in SAP AI Core allows users to make their AI-generated content available on 
the Service Marketplace. This feature leverages GitOps to streamline the deployment and management of AI 
content, making it accessible for other developers and businesses.
Title: Lifecycle Management in AI Core
Query: Why is lifecycle management important in SAP AI Core?
Answer: Lifecycle management is crucial in SAP AI Core as it ensures that AI models are effectively trained, 
deployed, and maintained. This process helps in managing updates, monitoring performance, and optimizing 
models over time, leading to improved reliability and effectiveness.
Title: How to Experiment with Natural Language Prompts
Query: What is the process for experimenting with natural language prompts in SAP AI Core?
Answer: To experiment with natural language prompts, you can access the Generative AI Hub within SAP AI 
Core. Here, you can create and run various prompts using different generative AI models, allowing you to test 
and refine your inputs for better outcomes.
Title: Supported AI Models in SAP AI Core
Query: What AI models are supported by SAP AI Core?
Answer: SAP AI Core supports a variety of AI models, including those from Azure OpenAI and GCP Vertex AI. 
This diverse support allows users to choose the most suitable models for their specific applications and use 
cases.
Title: Managing AI Assets in SAP AI Core
Query: What is the best way to manage AI assets in SAP AI Core?
Answer: The best way to manage AI assets in SAP AI Core is by utilizing the Artifacts feature, which allows you 
to organize and reference your datasets and model files efficiently. This structured approach helps in 
maintaining clarity and accessibility of your AI resources.
Title: The Importance of Prompt Lifecycle Management
Query: What is prompt lifecycle management and why is it important?
Answer: Prompt lifecycle management involves overseeing the creation, testing, and optimization of prompts 
used in generative AI models. It is important because it ensures that prompts are effective and relevant, 
leading to better model performance and user satisfaction.
Title: How to Access Generative AI Capabilities
Query: What steps do I need to take to access generative AI capabilities in SAP AI Core?
Answer: To access generative AI capabilities, you must activate the Generative AI Hub within SAP AI Core. 
Once activated, you can explore various models and tools for prompt experimentation and management.
Title: Best Practices for AI Model Deployment
Query: What are some best practices for deploying AI models in SAP AI Core?
Answer: Best practices for deploying AI models include thoroughly testing your models before deployment, 
monitoring performance metrics post-deployment, and ensuring that you have a rollback plan in case of 
issues. Additionally, keeping documentation up to date is essential for maintaining clarity.
Title: How to Provide Feedback on SAP AI Core Documentation
Query: Is there a way to provide feedback on the SAP AI Core documentation?
Answer: Yes, you can provide feedback on the SAP AI Core documentation through GitHub by creating an 
issue. This allows you to communicate any concerns or suggestions directly to the authors and development 
team.
Title: Understanding the AI API Overview
Query: Can someone summarize the AI API Overview in SAP AI Core?
Answer: The AI API Overview in SAP AI Core outlines the available APIs for integrating AI functionalities into 
applications. It provides developers with the necessary tools to leverage machine learning capabilities and 
streamline the development process.
Title: Exploring the Generative AI Hub Architecture
Query: What does the architecture of the Generative AI Hub look like in SAP AI Core?
Answer: The architecture of the Generative AI Hub in SAP AI Core includes components for prompt 
experimentation, management, and administrative tools. It allows users to create and run natural language 
prompts while managing collections and metadata for better organization and efficiency."""

Execute Queries for Each Model

We will iterate through the defined models, send each question, and store the results. 

In [11]:
# Dictionary to store the results
master = {}

# Iterate through all model names
for model_name in models:
    print(f":::{model_name}:::\n")
    
    master[model_name] = {
        "question": [],
        "answer": []
    }
    
    for query in questions:

        # Call Orchestration_run with query and context
        ans = Orchestration_run(query=query, context=context, model_name=model_name)
        
        master[model_name]["question"].append(query)
        master[model_name]["answer"].append(ans)
        
        print("QUERY: ", query + '?????\n')
        print("RESPONSE: ", ans + '\n')
        print()


:::gpt-4o:::

QUERY:  What are the primary benefits of using SAP AI Core for managing AI scenarios??????

RESPONSE:  The primary benefits of using SAP AI Core for managing AI scenarios are as follows:

**SAP AI Core is a service within the SAP Business Technology Platform that manages AI assets in a 
standardized and scalable manner. It supports the execution and operations of AI scenarios, allowing for 
seamless integration with SAP solutions. Its main functions include lifecycle management of AI scenarios, 
data-driven decision-making, and automation of tasks using machine learning capabilities.**


QUERY:  How does SAP AI Core facilitate the integration of AI models with other SAP solutions??????

RESPONSE:  **APIs in SAP AI Core facilitate the integration of AI capabilities into applications, allowing developers to access machine learning functionalities seamlessly. They provide standardized methods for interacting with AI models, making it easier to implement AI solutions in vario

Exporting Results to Excel

In this section, we convert the collected results from each model into a single DataFrame for easy export


In [12]:
# Convert results to a DataFrame for exporting
results_df = pd.DataFrame()

# Create a DataFrame for each model's results and concatenate them
for model_name in models:
    model_df = pd.DataFrame(master[model_name])
    model_df['model'] = model_name  # Add a column for the model name
    results_df = pd.concat([results_df, model_df], ignore_index=True)

# Save the results to an Excel file
results_df.to_excel("llm_responses.xlsx", index=False)